# Simularium Conversion Tutorial : Smoldyn Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio.smoldyn import SmoldynConverter, SmoldynData
from simulariumio import UnitData, MetaData
from simulariumio.filters import TranslateFilter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium `SmoldynConverter` consumes spatiotemporal data from Smoldyn. 

The converter requires a `SmoldynData` object as a parameter.

`SmoldynData` contains the following:
* **meta_data** : `MetaData`
    * An object containing metadata for the trajectory including box size, scale factor, and camera defaults  
* **path_to_output_txt** : *str*
    * A string path to the output txt file
    * Generate by adding to your config.txt file:
        `output_files output.txt
        cmd n 1 executiontime output.txt
        cmd n 1 listmols output.txt`          
* **radii** : *Dict\[str, float\] (optional)*
    * A mapping of type names to the radii with which to draw them
    * Default: 1.0 (for any type name not specified)
* **display_names** : *Dict\[str, str\] (optional)*
    * A mapping from original species names to names to display in the Simularium Viewer
    * Default: use original names
* **time_units:** `UnitData` *(optional)*
    * multiplier and unit name for time values
    * Default: 1.0 second            
* **spatial_units**: `UnitData` *(optional)*
    * multiplier and unit name for spatial values (including positions, radii, and box size)
    * Default: 1.0 meter           
* **plots** : *List\[Dict\[str, Any\]\] (optional)*
    * An object containing plot data already in Simularium format
    
`MetaData` contains the following:
* **box_size** : *np.ndarray (shape = \[3\])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
        * *for now we only support rectangular bounds, but will be adding support for spheres soon*
* **camera_defaults**: `CameraData` (optional)
    * camera's initial settings which it also returns to when reset
* **scale_factor** : float (optional)
    * A multiplier for the scene, use if visualization is too large or small
    * Default: 1.0
    
`CameraData` contains the following:
* **position** : *np.ndarray (shape = \[3\]) (optional)*
    * 3D position of the camera itself
    * Default: np.array(\[0.0, 0.0, 120.0\])
* **look_at_position**: *np.ndarray (shape = \[3\]) (optional)*
    * position at which the camera looks
    * Default: np.zeros(3)
* **up_vector**: *np.ndarray (shape = \[3\]) (optional)*
    * the vector that defines which direction is "up" in the camera's view
    * Default: np.array(\[0.0, 1.0, 0.0\])
* **fov_degrees**: *float (optional)*
    * the angle defining the extent of the 3D world that is seen from bottom to top of the camera view
    * Default: 50.0

`UnitData` contains the following:
* **name**: *str*
    * unit name for values (we support this list https://github.com/hgrecco/pint/blob/master/pint/default_en.txt)
* **magnitude**: *float (optional)*
    * multiplier for values (in case they are not given in whole units)
    * Default: 1.0

In [2]:
box_size = 100.

example_data = SmoldynData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
    ),
    path_to_output_txt="../simulariumio/tests/data/smoldyn/example_3D.txt",
    radii={
        "red(solution)" : 1.0,
        "green(solution)" : 2.0,
    },
    display_names={
        "red(solution)" : "B",
        "green(solution)" : "A",
    },
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

(since this model's coordinates are all positive, use a `TranslateFilter` to center the data in the viewer.)

In [3]:
c = SmoldynConverter(example_data)
translation_magnitude = -box_size / 2
filtered_data = c.filter_data([
    TranslateFilter(
        translation_per_type_id={},
        default_translation=translation_magnitude * np.ones(3)
    ),
])
c.write_external_JSON(filtered_data, "example_smoldyn")

Reading Smoldyn Data -------------
Filtering: translation -------------
Writing JSON (external)-------------
Converting Trajectory Data -------------
saved to example_smoldyn.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.